In [3]:
import xarray as xr
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from datetime import datetime

In [7]:
## Checking where the file has saved - it's in my OneDrive in Upwelling_PhD
import os
os.getcwd()

'C:\\Users\\dns21kxq'

In [9]:
os.chdir('C:\\Users\\dns21kxq\\OneDrive - Bangor University\\Upwelling_PhD')

In [11]:
## Open netcdf file
ds = xr.open_dataset("cmems_mod_glo_phy_my_0.083deg_P1M-m_mlotst_118.33E-289.42E_49.33S-56.58N_1993-01-01-2021-06-01.nc")

In [13]:
# Ensure time is in datetime format
ds['time'] = pd.to_datetime(ds['time'].values)

In [19]:
# Date range
start_date = "1993-01-01"
end_date = "2021-06-01"
times = pd.date_range(start=start_date, end=end_date, freq='MS')  # monthly start

In [23]:
!pip install haversine 

In [33]:
from haversine import haversine, Unit

def find_nearest_ocean_point_fast(lat, lon, ds, var='MLD'):
    # Convert 2D lat/lon grids
    lat_grid, lon_grid = np.meshgrid(ds.latitude.values, ds.longitude.values, indexing='ij')

    # Convert longitudes in dataset to [-180, 180] for haversine
    lon_grid = np.where(lon_grid > 180, lon_grid - 360, lon_grid)

    # Flatten and filter NaNs
    sample = ds[var].isel(time=0).values
    mask = ~np.isnan(sample)
    valid_lats = lat_grid[mask]
    valid_lons = lon_grid[mask]

    # Vectorized haversine distance calculation
    dists = np.array([
        haversine((lat, lon), (la, lo), unit=Unit.KILOMETERS)
        for la, lo in zip(valid_lats, valid_lons)
    ])

    idx = np.argmin(dists)
    
    # Get back 2D index
    flat_idx = np.flatnonzero(mask)[idx]
    lat_idx, lon_idx = np.unravel_index(flat_idx, sample.shape)
    return lat_idx, lon_idx


In [39]:
# Convert all input longitudes to [-180, 180]
missing_islands = {
    "Guam": (13.43556, 144.7694),
    "Hawaii": (19.65317, 204.4824 - 360),
    "Kauai": (22.02798, 200.4573 - 360),
    "Lanai": (20.82311, 203.0992 - 360),
    "Lisianski": (26.01765, 186.0592 - 360),
    "Maui": (20.8275, 203.5728 - 360),
    "Oahu": (21.46262, 202.0198 - 360),
    "Ofu & Olosega": (-14.17103, 190.3512 - 360),
    "Rota": (14.14488, 145.1785),
    "Tinian": (15.01948, 145.629)
}


In [41]:
import pandas as pd

all_islands_data = []

for island, (lat, lon) in missing_islands.items():
    lat_idx, lon_idx = find_nearest_ocean_point_fast(lat, lon, ds, var='mlotst')
    mld_timeseries = ds['mlotst'][:, lat_idx, lon_idx].to_series()
    mld_timeseries = mld_timeseries.reset_index()
    mld_timeseries['Island'] = island
    all_islands_data.append(mld_timeseries)


In [43]:
island_mld_filled = pd.concat(all_islands_data, ignore_index=True)


In [57]:
# Load island mean MLD dataset
df = pd.read_csv("island_mld_timeseries.csv", parse_dates=["Date"])

In [59]:
# Combine original island data with the filled-in missing island MLD data
combined_df = pd.concat([df, island_mld_filled], ignore_index=True)


In [61]:
combined_df.to_csv("island_mld_timeseries_filled.csv", index=False)


In [11]:
### Above code didn't work - still getting NA values

ds = xr.open_dataset('cmems_mod_glo_phy_my_0.083deg_P1M-m_mlotst_118.33E-289.42E_49.33S-56.58N_1993-01-01-2021-06-01.nc', decode_coords="all", mask_and_scale=True)

In [13]:
print(ds['mlotst'].attrs)  # Look for _FillValue or missing_value


{'unit_long': 'Meters', 'long_name': 'Density ocean mixed layer thickness', 'valid_max': 23400, 'units': 'm', 'valid_min': 1, 'standard_name': 'ocean_mixed_layer_thickness_defined_by_sigma_theta'}


In [17]:
ds['mlotst'].attrs


{'unit_long': 'Meters',
 'long_name': 'Density ocean mixed layer thickness',
 'valid_max': 23400,
 'units': 'm',
 'valid_min': 1,
 'standard_name': 'ocean_mixed_layer_thickness_defined_by_sigma_theta'}

In [48]:
### Instead, calculating a mean MLD value for each island covered by land, for each time step

import numpy as np
import pandas as pd
import xarray as xr

# Island coordinates (in decimal degrees; longitudes should be 0–360)
islands = {
    'Guam': (13.43556, 144.7694), 
    'Hawaii': (19.65317, 204.4824),
    'Kauai': (22.02798, 200.4573),
    'Lanai': (20.82311, 203.0992),
    'Lisianski': (26.01765, 186.0592),
    'Maui': (20.8275, 203.5728),
    'Oahu': (21.46262, 202.0198),
    'Ofu & Olosega': (-14.17103, 190.3512),
    'Rota': (14.14488, 145.1785),
    'Tinian': (15.01948, 145.629),
}

# Helper: find index of nearest coordinate
def find_nearest_idx(array, value):
    return np.abs(array - value).argmin()

# Loop over islands
island_timeseries = []

for island, (lat, lon) in islands.items():
    # Ensure longitude is 0–360
    if lon < 0:
        lon = lon + 360
    
    # Find nearest grid cell indices
    lat_idx = find_nearest_idx(ds.latitude.values, lat)
    lon_idx = find_nearest_idx(ds.longitude.values, lon)

    # Define a small window around the grid cell (e.g., 3x3 neighborhood)
    lat_slice = slice(max(lat_idx - 1, 0), min(lat_idx + 2, len(ds.latitude)))
    lon_slice = slice(max(lon_idx - 1, 0), min(lon_idx + 2, len(ds.longitude)))

    # Extract the 3x3 surrounding MLD values for each timestep
    mld_sub = ds['mlotst'][:, lat_slice, lon_slice]

    # Compute mean across spatial dims, ignoring NaNs
    mld_mean = mld_sub.mean(dim=['latitude', 'longitude'], skipna=True)

    # Turn into DataFrame
    island_df = mld_mean.to_dataframe(name='MLD').reset_index()
    island_df['Island'] = island

    island_timeseries.append(island_df)

# Combine all islands
result_df = pd.concat(island_timeseries, ignore_index=True)

# Preview
print(result_df.head())


        time        MLD Island
0 1993-01-01  41.924804   Guam
1 1993-02-01  35.897398   Guam
2 1993-03-01  25.902586   Guam
3 1993-04-01  22.392957   Guam
4 1993-05-01  18.768884   Guam


In [54]:
result_df.to_csv("MLD_timeseries_by_island_FILLED.csv", index=False)

## Worked for all except Hawaii and Oahu - next, solution for these 2 islands using bigger grid area surrounding islands

In [30]:
import numpy as np
import pandas as pd

def extract_mld_5x5(ds, lat, lon, island_name):
    lat_grid, lon_grid = np.meshgrid(ds.latitude.values, ds.longitude.values, indexing='ij')
    lon_grid = np.where(lon_grid < 0, lon_grid + 360, lon_grid)  # Normalize longitude if needed
    
    # Find nearest grid cell indices
    lat_idx = (np.abs(ds.latitude.values - lat)).argmin()
    lon_idx = (np.abs(lon_grid[0] - lon)).argmin()

    # Define window size and half window
    window_size = 5
    half_window = window_size // 2

    # Initialize list to hold mean MLD for each timestep
    mean_mld = []

    # Loop through time dimension
    for t in range(len(ds.time)):
        # Extract 5x5 window slice with bounds check
        lat_min = max(0, lat_idx - half_window)
        lat_max = min(len(ds.latitude) - 1, lat_idx + half_window)
        lon_min = max(0, lon_idx - half_window)
        lon_max = min(len(ds.longitude) - 1, lon_idx + half_window)

        window_data = ds['mlotst'][t, lat_min:lat_max+1, lon_min:lon_max+1].values
        window_mean = np.nanmean(window_data)
        mean_mld.append(window_mean)
    
    # Create DataFrame for this island
    df = pd.DataFrame({
        'Date': ds.time.values,
        'Island': island_name,
        'MLD_5x5_mean': mean_mld
    })

    return df

# Coordinates for the two islands
islands_5x5 = {
    'Hawaii': (19.65317, 204.4824),
    'Oahu': (21.46262, 202.0198)
}

# Collect results
results = []

for island, (lat, lon) in islands_5x5.items():
    island_df = extract_mld_5x5(ds, lat, lon, island)
    results.append(island_df)

# Combine into single DataFrame
mld_5x5_df = pd.concat(results, ignore_index=True)

# Now you can save or merge this data with your existing dataframe
print(mld_5x5_df.head())


        Date  Island  MLD_5x5_mean
0 1993-01-01  Hawaii           NaN
1 1993-02-01  Hawaii           NaN
2 1993-03-01  Hawaii           NaN
3 1993-04-01  Hawaii           NaN
4 1993-05-01  Hawaii           NaN


C:\Users\dns21kxq\AppData\Local\Temp\ipykernel_6160\3760519329.py:28: RuntimeWarning: Mean of empty slice
  window_mean = np.nanmean(window_data)


In [32]:
# Check if the nearest grid cell is on NaN for Hawaii and Oahu

lat_idx = (np.abs(ds.latitude.values - 19.65317)).argmin()
lon_idx = (np.abs(ds.longitude.values - 204.4824)).argmin()
print("Nearest grid cell values for Hawaii at time 0:", ds['mlotst'][0, lat_idx, lon_idx].values)

## Need bigger grid cell window - trying 7x7


Nearest grid cell values for Hawaii at time 0: nan


In [42]:
# Islands dictionary for just Hawaii and Oahu, if you want
islands_7x7 = {
    'Hawaii': (19.65317, 204.4824),
    'Oahu': (21.46262, 202.0198)
}

results = []
for island, (lat, lon) in islands_7x7.items():
    df = extract_mld_window(ds, lat, lon, island, window_size=9)  # window_size=9 here
    results.append(df)

mld_df_9x9 = pd.concat(results, ignore_index=True)
print(mld_df_9x9.head())


        Date  Island  MLD_9x9_mean
0 1993-01-01  Hawaii     56.001468
1 1993-02-01  Hawaii     49.134803
2 1993-03-01  Hawaii     32.959992
3 1993-04-01  Hawaii     20.447402
4 1993-05-01  Hawaii     18.768884


In [44]:
mld_df_9x9.to_csv("MLD_timeseries_Hawaii_Oahu_FILLED.csv", index=False)